In [ ]:
import numpy as np

class AntColonyOptimization:
    def __init__(self, num_ants, num_iterations, alpha, beta, evaporation_rate, pheromone_deposit, initial_pheromone):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone_deposit = pheromone_deposit
        self.initial_pheromone = initial_pheromone

    def initialize_pheromones(self, num_cities):
        self.pheromones = np.full((num_cities, num_cities), self.initial_pheromone)

    def calculate_probabilities(self, visibility, pheromones, current_city, visited):
        probs = np.zeros(len(visibility))
        denominator = 0
        for i in range(len(visibility)):
            if i not in visited:
                denominator += (pheromones[current_city, i] ** self.alpha) * (visibility[current_city, i] ** self.beta)
        for i in range(len(visibility)):
            if i not in visited:
                numerator = (pheromones[current_city, i] ** self.alpha) * (visibility[current_city, i] ** self.beta)
                probs[i] = numerator / denominator
        return probs

    def construct_solution(self, distances, visibility):
        num_cities = len(distances)
        solutions = []
        for _ in range(self.num_ants):
            solution = []
            visited = set()
            current_city = np.random.randint(0, num_cities)
            solution.append(current_city)
            visited.add(current_city)
            while len(visited) < num_cities:
                probs = self.calculate_probabilities(visibility, self.pheromones, current_city, visited)
                next_city = np.random.choice(range(num_cities), p=probs)
                solution.append(next_city)
                visited.add(next_city)
                current_city = next_city
            solutions.append(solution)
        return solutions

    def calculate_solution_cost(self, solution, distances):
        cost = 0
        for i in range(len(solution) - 1):
            cost += distances[solution[i]][solution[i+1]]
        cost += distances[solution[-1]][solution[0]]  # Return to starting city
        return cost

    def update_pheromones(self, solutions, distances):
        self.pheromones *= (1 - self.evaporation_rate)
        for solution in solutions:
            cost = self.calculate_solution_cost(solution, distances)
            for i in range(len(solution) - 1):
                self.pheromones[solution[i]][solution[i+1]] += self.pheromone_deposit / cost
            self.pheromones[solution[-1]][solution[0]] += self.pheromone_deposit / cost

    def run(self, distances):
        num_cities = len(distances)
        visibility = 1 / (distances + np.eye(num_cities))  # Use the inverse of distance as visibility
        self.initialize_pheromones(num_cities)
        best_solution = None
        best_cost = float('inf')

        for _ in range(self.num_iterations):
            solutions = self.construct_solution(distances, visibility)
            self.update_pheromones(solutions, distances)
            for solution in solutions:
                cost = self.calculate_solution_cost(solution, distances)
                if cost < best_cost:
                    best_cost = cost
                    best_solution = solution

        return best_solution, best_cost

# Example usage
if __name__ == "__main__":
    distances = np.array([
        [0, 2, 2, 5, 7],
        [2, 0, 4, 8, 2],
        [2, 4, 0, 1, 3],
        [5, 8, 1, 0, 2],
        [7, 2, 3, 2, 0]
    ])
    aco = AntColonyOptimization(num_ants=10, num_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, pheromone_deposit=100, initial_pheromone=0.1)
    best_solution, best_cost = aco.run(distances)
    print("Best solution:", best_solution)
    print("Best cost:", best_cost)

Best solution: [2, 3, 4, 1, 0]
Best cost: 9


In [ ]:
import pandas as pd
import numpy as np

# Load the CSV files
cities_coordinates = pd.read_csv('Geocoded_Bangalore.csv')
cities_labels = pd.read_csv('clustered_data_BIRCH.csv')


In [ ]:
# Assuming both dataframes have a 'city' column to merge on
cities_data = pd.merge(cities_coordinates, cities_labels, on='Area')


In [ ]:
scarcity_cities = cities_data[cities_data['Cluster'] == 0]
surplus_cities = cities_data[cities_data['Cluster'] == 2]


In [ ]:
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

# Initialize the distance matrix
distance_matrix = np.zeros((len(surplus_cities), len(scarcity_cities)))

# Populate the distance matrix
x,y=0,0
for i, surplus_city in surplus_cities.iterrows():
    for j, scarcity_city in scarcity_cities.iterrows():
        distance_matrix[x, y] = haversine(surplus_city['Latitude'], surplus_city['Longitude'],
                                          scarcity_city['Latitude'], scarcity_city['Longitude'])
        y+=1
    x+=1
    y=0

print(distance_matrix)


[[2.99983589e+02 1.13916582e+04 1.15220327e+02 ... 2.94524789e+02
  3.04179119e+02 3.08075174e+02]
 [3.46636965e+02 1.14783978e+04 2.43133779e+02 ... 2.94509676e+02
  3.56839821e+02 3.24354231e+02]
 [1.92024476e+02 1.15835570e+04 2.66318014e+02 ... 1.18026970e+02
  2.06626222e+02 1.51331398e+02]
 ...
 [1.67716697e+00 1.15698654e+04 2.91689903e+02 ... 1.19643640e+02
  1.64437885e+01 8.01097094e+01]
 [1.64857557e+01 1.15547850e+04 2.76865187e+02 ... 1.27082671e+02
  1.66861594e+01 9.04555846e+01]
 [3.90218207e+01 1.15976308e+04 3.02508788e+02 ... 7.90584721e+01
  5.71370184e+01 4.03836046e+01]]


In [ ]:
distance_df = pd.DataFrame(distance_matrix, index=surplus_cities['Area'], columns=scarcity_cities['Area'])
print(distance_df)


Area         ISRO Layout  Old Airport Road      Hospet      Terdal  \
Area                                                                 
Byadgi        299.983589      11391.658162  115.220327  205.092153   
Bhatkal       346.636965      11478.397753  243.133779  283.645606   
Sakleshpur    192.024476      11583.557045  266.318014  402.688239   
Indiranagar    12.338280      11561.769671  288.586069  480.613898   
Anekal         26.034749      11590.413393  317.704196  508.490051   
...                  ...               ...         ...         ...   
Mudigere      209.168813      11563.825924  250.432095  379.080920   
Udupi         308.293621      11548.404565  277.516072  351.927401   
Ilyas Nagar     1.677167      11569.865441  291.689903  482.311502   
Jalahalli      16.485756      11554.784953  276.865187  468.297152   
Anjanapura     39.021821      11597.630778  302.508788  486.462382   

Area         Dodballapur  Chikballapur  Tumkur Road  Vijayanagar    Majestic  \
Area     

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Assuming distance_matrix is your NxM matrix where N is the number of surplus cities and M is the number of scarcity cities
# Ensure your distance matrix is in the form of a numpy array

# Using the linear sum assignment method (Hungarian algorithm) from scipy
row_ind, col_ind = linear_sum_assignment(distance_matrix)

# row_ind contains the indices of the surplus cities
# col_ind contains the indices of the assigned scarcity cities

# Now you can print the optimal assignments and the corresponding distances
total_distance = distance_matrix[row_ind, col_ind].sum()

print("Optimal assignment:")
for surplus_city_idx, scarcity_city_idx in zip(row_ind, col_ind):
    print(f"Surplus city {surplus_city_idx} -> Scarcity city {scarcity_city_idx} (Distance: {distance_matrix[surplus_city_idx, scarcity_city_idx]} km)")

print(f"Total distance: {total_distance} km")


Optimal assignment:
Surplus city 0 -> Scarcity city 31 (Distance: 13.784870272669957 km)
Surplus city 1 -> Scarcity city 49 (Distance: 51.19067665144735 km)
Surplus city 2 -> Scarcity city 33 (Distance: 63.740232051622286 km)
Surplus city 3 -> Scarcity city 12 (Distance: 0.5692049440781095 km)
Surplus city 5 -> Scarcity city 4 (Distance: 25.254955750458784 km)
Surplus city 6 -> Scarcity city 37 (Distance: 5.457065004528743 km)
Surplus city 8 -> Scarcity city 22 (Distance: 36.32053209722094 km)
Surplus city 12 -> Scarcity city 36 (Distance: 76.17758043494142 km)
Surplus city 13 -> Scarcity city 9 (Distance: 1.4202948072345238 km)
Surplus city 14 -> Scarcity city 17 (Distance: 3.8152839184687335 km)
Surplus city 15 -> Scarcity city 5 (Distance: 15.167707082600002 km)
Surplus city 16 -> Scarcity city 54 (Distance: 22.124062151206235 km)
Surplus city 20 -> Scarcity city 3 (Distance: 31.178775871747543 km)
Surplus city 21 -> Scarcity city 29 (Distance: 4.5143976077050585 km)
Surplus city 22

In [29]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 37.0 MB/s eta 0:00:00


In [30]:
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Assuming distance_matrix is already defined as a numpy array
num_surplus_cities, num_scarcity_cities = distance_matrix.shape

# Create the MILP problem
prob = LpProblem("OptimalAssignment", LpMinimize)

# Define decision variables
x = LpVariable.dicts("x", [(i, j) for i in range(num_surplus_cities) for j in range(num_scarcity_cities)],
                     cat='Binary')

# Objective function: Minimize total distance
prob += lpSum(x[i, j] * distance_matrix[i, j] for i in range(num_surplus_cities) for j in range(num_scarcity_cities))

# Constraints
# Each scarcity city must be visited exactly once
for j in range(num_scarcity_cities):
    prob += lpSum(x[i, j] for i in range(num_surplus_cities)) == 1

# Solve the problem
prob.solve()

# Print the results
print("Optimal assignment:")
total_distance = 0
for i in range(num_surplus_cities):
    for j in range(num_scarcity_cities):
        if value(x[i, j]) > 0.5:  # Check if the variable is selected in the optimal solution
            print(f"Surplus city {i} -> Scarcity city {j} (Distance: {distance_matrix[i, j]} km)")
            total_distance += distance_matrix[i, j]

print(f"Total distance: {total_distance} km")


Optimal assignment:
Surplus city 0 -> Scarcity city 31 (Distance: 13.784870272669957 km)
Surplus city 1 -> Scarcity city 49 (Distance: 51.19067665144735 km)
Surplus city 3 -> Scarcity city 12 (Distance: 0.5692049440781095 km)
Surplus city 5 -> Scarcity city 6 (Distance: 11.890194944258305 km)
Surplus city 6 -> Scarcity city 4 (Distance: 17.97493348638961 km)
Surplus city 6 -> Scarcity city 37 (Distance: 5.457065004528743 km)
Surplus city 8 -> Scarcity city 22 (Distance: 36.32053209722094 km)
Surplus city 12 -> Scarcity city 23 (Distance: 96.40651044080693 km)
Surplus city 12 -> Scarcity city 36 (Distance: 76.17758043494142 km)
Surplus city 13 -> Scarcity city 8 (Distance: 2.0351530498785038 km)
Surplus city 13 -> Scarcity city 9 (Distance: 1.4202948072345238 km)
Surplus city 13 -> Scarcity city 20 (Distance: 3.7650146561962834 km)
Surplus city 14 -> Scarcity city 15 (Distance: 2.083720267742117 km)
Surplus city 14 -> Scarcity city 17 (Distance: 3.8152839184687335 km)
Surplus city 15 ->